# Module 4b: TF-IDF vectorization

Een bron van veel informatie over de inhoud van films is de ondertiteling. In deze submodule gaan we kijken hoe we hier gebruik van kunnen maken. Via websites zoals [deze](https://www.opensubtitles.org/) kan je de ondertitels van films downloaden. Om tijd te besparen (en omdat 7000+ ondertitels downloaden niet zo'n leerzame bezigheid is) hebben wij dit vast gedaan.

Om iets nuttigs te kunnen doen met deze ondertitels zal je de ingesloten tekstuele informatie op de een of ander manier moeten structureren. Het structureren van natuurlijke tekst heet Natural Language Processing (NLP). NLP is een heel vakgebied op zichzelf waar we ons nu slechts deels in gaan verdiepen. Gelukkig is er spaCy, een Python package die NLP toegankelijk maakt voor onderzoekers die vooral bezig zijn met het verwerken van data. Dit gebeurt door veel keuzes voor aanpakken/algoritmes al voor de gebruiker te maken.

SpaCy komt met een grote verzameling aan nuttige tools voor het analyseren van tekst. Wij zullen het in deze submodule vooral gebruiken voor het _tokenizen_ (opsplitsen in losse woorden) van tekst.

We hebben ondertitelfiles bestaande uit vele regels tekst. In deze teksten staat nog een hoop rommel die voor ons niet interessant is; hoofdletters, leestekens, html tags, enters, etc. Het opdelen van de tekst in losse woorden heet _tokenization_. Dit is een fundamentele stap in bijna alle NLP taken. Nu hebben jullie al kennis gemaakt met Reguliere Expressies en zouden jullie dit in theorie daarmee kunnen afhandelen, maar met spaCy gaat dit allemaal een stuk eenvoudiger.

## Installatie

SpaCy is al een onderdeel van de conda CI enviromnet die je aan het begin van dit vak hebt geïnstalleerd. Dus dat hoef je niet meer te installeren. Maar spaCy werkt met taalmodellen die je los moet downloaden. Dat moet je nog wel doen. Voor het downloaden van de taalmodellen van het Engels, typ de volgende commando's in je terminal:

    python -m spacy download en_core_web_sm
    python -m spacy download en_core_web_md


## Oefeningen

Doorloop het eerste hoofdstuk van de spaCy-cursus: [spaCy: Finding words, phrases, names and concepts](https://course.spacy.io/chapter1)


## Achtergrond

Veel vormen van content-based filtering werken uiteindelijk toe naar het maken van een similarity matrix voor items. Zoals we bij collaborative filtering (en content-based filtering aan de hand van genres) hebben gezien, zodra je weet in welke mate items (films) op elkaar lijken, kan je aan de hand daarvan aanbevelingen doen. 

In alle gevallen die we tot nu toe hebben gezien maakten we een utility matrix. In deze matrix komt elke film overeen met een serie getallen. In het geval van collaborative filtering waren dit de ratings. In het geval van content-based filtering aan de hand van genres, waren dit de eentjes een nulletjes die aangaven of een film een bepaald genre had of niet.

Het assicieren van een item met een reeks getallen wordt *vectorisatie* genoemd. (Zo'n reeks getallen wordt ook wel een vector genoemd.) We hebben dus twee vormen van vectorisatie van films gezien: Bij collaborative filtering is elke film een vector met ratings. Bij content-based filtering is elke film een vector met eentjes en nulletjes (gegeven een genre).

Door de data op deze manier te vectoriseren kunnen we similatity maten zoals de cosine similarity en de jaccard index om een similarity matrix te maken.

Ook als we films met elkaar willen vergelijken aan de hand van de ondertiteling willen we uiteindelijk zo'n vectorisatie maken. Dus we willen de ondertiteling omzetten in een reeks getallen die we met elkaar kunnen vergelijken. Dat kan op meerdere manieren. In deze submodule gebruiken we daar *TF-IDF vectorisatie* voor. In een volgende module ga je nog naar een andere manier kijken.

TF-IDF staat voor *term frequency - inverse document frequency*. Het idee is dat we voor elk woord in een tekst een score willen geven. De score geeft aan hoe representatief de score voor de tekst is. Met ander woorden een hoge score betekent dat het gegeven woord veel vaker in de tekst voorkomt dan in andere teksten. Hoe je die score precies bepaald kom je in deze opdracht achter.

Door voor elk woord zo'n score te genereren kunnen we teksten met elkaar vergelijken aan de hand van de betreffende scores.

## Opdracht

Begin met het laden van de benodige libraries.

In [ ]:
import re
import spacy
import pandas as pd
import numpy as np
import sklearn
import os
import answers
from tqdm import tqdm

## Data laden

We laden eerst tien willekeurige ondertitelingen om mee te experimenteren.

In [ ]:
txt_path = "./data/txts"

# load 10 arbitrary files
files = os.listdir(txt_path)[120:130]

De data bestaat nu uit grote lappen tekst. Het eerste wat we moeten doen is de tekst opbreken in losse woorden (tokenizing). Hieronder tokenizen we de ondertitelingen en slaan we het resultaat op in de Series `movie_subs`.

In [ ]:
# Laad het taalmodel voor Engels
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


movie_ids = []
movie_subs = []

# Tokenize ieder van de losse ondertitelfiles
for ix, file in enumerate(files):
    with open(f'{txt_path}/{file}', 'r', encoding='iso-8859-1') as sub_file:
        # Haal de movieID uit de filenaam
        movie_ids.append(int(file.split('_')[0]))
        
        # Haal alle HTML tags zoals <i></i> weg.
        content = sub_file.read()
        content = re.sub(r"<[^>]*>", "", content)
        
        # Lees, verwerk, sla op...
        doc = nlp(content)
        movie_subs.append(doc)
        
# ... en maak de series
movie_subs = pd.Series(index=movie_ids, data=movie_subs)
display(movie_subs)

Als je wilt weten welke films dit precies zijn kan je de indices terugvinden in het `movies.csv` bestand:

In [ ]:
datapath = "./data/ml-latest-small"

# df_movies.loc[193581]
df_movies = pd.read_csv(f"{datapath}/movies.csv", index_col=0)
display(df_movies.loc[movie_subs.index])

## Bag of Words

Nu we alle ondertitelingen hebben getokenized met spaCy kunnen we bij alle onderdelen van de tekst. Nu weten we ook met redelijke zekerheid wat de werkwoorden zijn, wat de lidwoorden of wat helemaal geen woord is maar bijvoorbeeld punctuatie.

We zijn aan het werken aan een classificatie systeem op basis van de gebruikte woorden in de tekst. Nu geeft spaCy standaard een `Document` terug (een lijst van `Token`s). Omdat we vooral geïnteresseerd zijn in de frequentie van woorden in een tekst is dit een onhandig formaat. Elke keer als je wilt weten hoe vaak een woord in een document voorkomt zou je het het hele document moeten doorzoeken. Dus kunnen we beter een andere structuur gebruiken: een _bag of words_.

Een _bag of words_ is niks meer dan een datastructuur die per woord bijhoudt hoe vaak deze voorkomt in een tekst. Het ligt voor de hand om hier `Series` voor te gebruiken in Pandas. Een `Series` met als index de woorden en als waardes de hoeveelheid keer dat een woord voorkomt in de tekst. Zo kan je straks bijvoorbeeld heel snel [uitvinden](https://en.wikipedia.org/wiki/List_of_films_that_most_frequently_use_the_word_%22fuck%22) welke films sowieso een "PG-13 rating" of "R-rating" hebben.

### Vraag 1

\[1 pt.\]

Implementeer de functie `bag_of_words` hieronder.

In [ ]:
def lemmas(doc):
    """
    Returns a list of lemmas 
    """
    return [word.lemma_ for word in doc if word.is_alpha and not (word.is_punct or word.is_space or word.is_stop)]

def bag_of_words(doc):
    """
    Takes a spaCy doc and returns a bag of words.
    Does not include any punctuation, whitespace or stop words in the resulting bag.
    """
    # TODO
    
    
# Bag of words for The Conjuring
print(bag_of_words(movie_subs.loc[103688]).sort_values(ascending = False))

# Bag of words for The Lawnmower Man
print(bag_of_words(movie_subs.loc[1037]).sort_values(ascending = False))

In [ ]:
answers.test_7(bag_of_words)

## Tijdbesparing

Het berekenen van de bag of words voor alle 7149 films waarvoor we de ondertitels hebben kost erg veel tijd. Opdat je jouw computer niet uren hoeft te laten rekenen, hebben we alle bag of words voor alle subtitles gegenereerd en meegeleverd. Onderstaande stukje code laadt deze in en slaat ze op in een `Series` genaamd bags. Dit kan een paar minuten duren.

In [ ]:
bag_path = "./data/bags"

# Find all bag of words files
filenames = os.listdir(bag_path)

movie_ids = []
bags = []

# Tokenize ieder van de losse ondertitelfiles
for ix, filename in enumerate(tqdm(filenames)):
    path = f'{bag_path}/{filename}'
    movie_ids.append(int(filename.split(".")[0]))
    # Lees, verwerk, sla op...
    bag = pd.read_pickle(path)
    bags.append(bag)
        
# ... en maak de series
bags = pd.Series(index=movie_ids, data=bags)

## Term Frequency
Welke termen (woorden) komen er het meeste voor in een film? Dit kunnen we nu halen uit de bag of words. Dit is bijvoorbeeld de top 20 voor de film Die Hard:

In [ ]:
die_hard = bags[1036]
die_hard_sorted = die_hard.sort_values(ascending=False)
display(die_hard_sorted[:20])

Als we willen weten hoe representatief een term is voor een bepaalde film zegt alléén het aantal aantal voorkomens nog niet zo veel. Dit hangt onder andere af van de lengte van de film of de hoeveelheid aan dialoog. Laten we daarom kijken naar hoe vaak een term is gebruikt ten opzichte van het totaal aantal termen. Dit noemen we de _term frequency_

Laten we de ondertiteling file van Die Hard even document $d$ noemen. In dit document komt de term ($t$) John 37 keer voor. En het document bevat in totaal 3549 woorden. In dat geval kunnen we de term frequency $t$ in document $d$ als volgt uitrekenen:

$$
\textrm{tf}(t,d) = \frac{37}{3549} = 0.0104
$$

De formele definitie voor de term frequency $\textrm{tf}(t, d)$ van term $t$ in document $d$:
$$
\textrm{tf}(t, d) = \frac{\textrm{f}_{t, d}}{\sum_{t'\in d} \textrm{f}_{t',d}}
$$

Waar $\textrm{f}_{t, d}$ het aantal keer is dat term $t$ in document $d$ voorkomt.

### Vraag 2

\[1 pt.\]

Implementeer de functie `term_frequency` hieronder.

In [ ]:
def term_frequency(bag):
    """Takes in a bag of words, returns the term frequency of every term in bag as a Series."""
    # TODO
    

In [ ]:
answers.test_8(doc, bag_of_words, term_frequency, nlp)

### Inverse Document Frequency

Alleen de TF van eeen woord voor een document zegt ook nog niet zo veel. Woorden die over het algemeen vaak voorkomen, zoals `come` en `go`, zeggen niet zo heel veel over de film Die Hard, maar ze hebben wel een hoge TF. Om deze eruit te filteren kunnen we gebruik maken van de Inverse Document Frequency (IDF). De IDF is een waarde voor een woord die laag is als het woord in veel documenten voorkomt. Hierbij kijken we naar alle documenten in de dataset.

Met andere woorden de IDF is een maat die aangeeft hoe _onderscheidend_ een term is gegeven een verzameling van documenten (films in ons geval). Hoeveel documenten zijn er? En in hoeveel daarvan komt de term in kwestie voor? 

Stel dat de volledige verzameling $D$ uit 7154 documenten bestaat en de term $t$ John in 1897 van die documenten voorkomt, dan is de IDF:

$$
\textrm{idf}(t, D) = \log(\frac{7154}{1897}) \approx 1.327
$$

De formele definitie is:

$$
\textrm{idf}(t, D) = \log(\frac{|D|}{|D_t|})
$$

Waarbij $|D|$ het totaal aantal documenten is en $|D_t|$ het aantal documenten met term $t$ erin.

N.B. In deze formule staat een _log_. Er zijn een aantal goede redenen voor om hier een log te gebruiken, maar die gaan voorbij de scope van deze opdracht. Als je hier toch graag meer over wilt weten, dan kan je je storten op [dit artikel](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.438.2284&rep=rep1&type=pdf). Dit is een vrij technisch artikel en je hebt niet alle wiskunde en statistiek gehad om alles te kunnen volgen, maar het kan toch leerzaam zijn het eens door te lezen.

### Vraag 3

\[1 pt.\]

Implementeer nu de functie `inverse_document_frequency` hieronder.

**Let op** Afhankelijk van hoe je `inverse_document_frequency` implementeert kan het best lang duren om de IDF voor alle termen uit te rekenen. Je kan daarom beter debuggen met een subset van alle bags, zeg 10 of 100 bags. Ook hebben wij de IDF voor alle termen al uitgerekend en meegeleverd, deze wordt in een latere cel ingeladen. Je hoeft dus de IDF voor alle termen niet zelf te berekenen.

In [ ]:
# TODO


def inverse_document_frequency(bags):
    """Takes in a Series of bag of words, returns the IDF for every term in the bags in a Series."""
    # TODO
    
    
    
idf = inverse_document_frequency(bags[120:130])
print(idf.sort_values())

In [ ]:
answers.test_9(nlp, bag_of_words, inverse_document_frequency)

## Tijdbesparing
We hebben de IDF voor alle termen al voor je uitgerekend. Zo hoef je niet op je computer te wachten. Onderstaande stukje code laadt de IDF Series in.

In [ ]:
idf = pd.read_pickle('./data/idf.pkl')
display(idf.sort_values())

## TF-IDF
Term frequency en inverse document frequency kunnen we combineren. Zo krijgen we een score voor hoe relevant een term is voor een document (film). 

In het voorbeeld dat we hierboven hebben gebruikt is de TF voor John ($t$) in het subtitle document ($d$) Die Hard ongeveer 0.0104 en de IDF voor het woord John in onze corpus ($D$) ongeveer 1.327. De TF-IDF score is dan simpelweg de twee vermenigvuldigd:

$$
\textrm{tf-idf}(t, d, D) \approx 0.0104 * 1.327 \approx 0.0138
$$

De formele definitie:

$$
\textrm{tf-idf}(t, d, D) = \textrm{tf}(t, d) * \textrm{idf}(t, D)
$$

### Vraag 4

\[1 pt.\]

Implementeer `tf_idf` hieronder:

In [ ]:
def tf_idf(bag, idf):
    """
    Takes in a bag of words and a Series containing IDF scores, 
    returns tf_idf score for every word in bag as a Series
    """
    # TODO
    
    
tf_idf(bags[1036], idf).sort_values(ascending=False)[:20]

In [ ]:
answers.test_11(nlp, bag_of_words, tf_idf)

### Vraag 5

\[2 pt.\]

Kijk naar de resultaten van `tf_idf`. Je ziet een lijst met woorden die duidelijk karakteristiek zijn voor de film Die Hard. Het zou alleen kunnen dat er woorden tussen zitten die niet nuttig zijn voor het vergelijken met andere films. Bijvoorbeeld "McClane" is het hoofpersonage van Die Hard. Die naam is heel karakterisitiek voor de film, maar het is geen nuttig woord als we willen weten of Die Hard op een andere film lijkt. Waarom niet? En zou je een manier kunnen verzinnen om dat soort woorden te vermijden? (Je hoeft dit niet te implementeren!)

YOUR ANSWER HERE

### Vraag 6

\[1 pt.\]

Het bijhouden van een volledige `bag_of_words` gaat ten koste van de snelheid van veel algoritmen. In een praktijk, met geavanceerde hardware, hoeft dat geen probleem te zijn, maar het kan lonen om de `bag_of_words` kleiner te maken. Gelukkig zijn we in de praktijk vooral geinterreseerd in bepaalde delen van de `bag_of_words`. Hoe zouden we de `bag_of_words` kunnen reduceren met zoveel mogelijk behoud van effectiviteit? (Er is hier niet een enkel goed antwoord, het gaat om de redenatie)

YOUR ANSWER HERE

## Vectorizer

Je hebt nu een manier om films te vectoriseren. De output van de `tf_idf`-functie _is_ de vectorisatie van een film. Hiermee kan je een utility matrix maken. Het kan trouwens hiervoor handig zijn om niet de `tf-idf` score te gebruiken van _alle_ woorden, maar bijvoorbeeld alleen de top-N van woorden met de laagste IDF score.

Je krijgt dan een utility matrix (vectorisatie) die er ongeveer zo uitziet:

<img src = "./include/tf-idf-vectorization.png" width = 70%>

De features zijn de woorden en de waardes zijn de tf-idf scores voor de betreffend film.

Als je de matrix beperkt tot de top-N woorden met de laagste IDF score, is het handig om eerst de stopwoorden er uit te filteren. Stopwoorden zijn woorden die heel veel voorkomen (en dus een hele lage IDF score hebben), maar weinig informatie toevoegen (woorden zoals "so", "why", "yes", ...).

spaCy heeft een ingebouwde lijst met stopwoorden genaamd `STOP_WORDS`. Die kan je zo gebruiken:

In [ ]:
from spacy.lang.en import STOP_WORDS

if 'why' in STOP_WORDS:
    print('"why" is a stop word')
else:
    print('"why" is not a stop word')
    
if 'supercalifragilisticexpialidocious' in STOP_WORDS:
    print('"supercalifragilisticexpialidocious" is a stop word')
else:
    print('"supercalifragilisticexpialidocious" is not a stop word')

### Vraag 7

\[2 pt.\]

Schrijf hieronder de functie `tf_idf_vectorizer`. Deze functie heeft als input:

- `bags`, de bag of words van alle films
- `idf`, de idf scores van alle woorden voor de hele dataset
- `max_features`, het maximale aantal woorden om te gebruiken als features (top-N met laagste idf score)
- `stopwords`, woorden die je mag negeren (deze tellen dus ook niet mee voor de max-features)

De ouput is een dataframe met de films als rijen, de woorden als kolommen en de tf-idf scores als waardes. Zie ook het plaatje hierboven.

- De `idf` input geeft een reeks van woorden met IDF waardes. De eerste stap is een subselectie maken door alle `stopwords` uit de lijst te halen en vervolgens de top-N woorden met laagste idf score te selecteren. Hoeveel woorden de top-N bevat wordt gegeven door `max_features`. Implementeer en test dit deel eerst. 

- Je moet vorvolgens in de functie voor elke film en voor elk woord de tf_idf score berkenen. Je kan hier natuurlijk de `tf_idf`-functie gebruiken die je bij vraag 4 hebt geïmplementeerd.

De functie wordt hieronder aangeroepen op slechts een kleine selectie van films. De tf-idf vectorisatie bereken voor de hele dataset zou onpraktisch veel tijd kosten.

In [ ]:
def tf_idf_vectorizer(bags, idf, max_features = 500, stopwords = set()):
    # TODO
    

# load set with stop words from spaCy
from spacy.lang.en import STOP_WORDS

# selection of movies to vectorize (the entire dataset will take way too much time)
selection = [1032, 1036, 103253, 103335, 103341, 106487, 106782, 109864, 110, 110102, 1103, 117529, 1222, 122882]

# apply function
tf_idf_vectorization = tf_idf_vectorizer(bags.loc[selection], idf, max_features = 1000, stopwords = STOP_WORDS)
display(tf_idf_vectorization)

Run de code hieronder om te zien welke films je zojuist hebt gevectoriseert

In [ ]:
datapath = "./data/ml-latest-small"
df_movies = pd.read_csv(f"{datapath}/movies.csv", index_col=0)
display(df_movies.loc[tf_idf_vectorization.index, 'title'])

Test je uitwerking:

In [ ]:
answers.test_12(tf_idf_vectorizer)

Is dit een goede uitkomst? Dat is zo op het oog natuurlijk niet meteen te zeggen. We kunnen wel even als sanity check kijken wat het woord is met de hoogste tf-idf score voor elke film en kijken of dat logisch lijkt:

In [ ]:
# link vectorization to movie titles
tf_idf_vectorization_with_titles = tf_idf_vectorization.copy()
tf_idf_vectorization_with_titles.index = df_movies.loc[tf_idf_vectorization.index, 'title']

# highest scoring feature
best_feature = tf_idf_vectorization_with_titles.idxmax(axis=1)
print(best_feature)

### Vraag 8
\[2 pt.\]

Maak hieronder een cosine similarity matrix voor deze vectorisatie. Je kan hiervoor de functie `create_similarity_matrix_cosine` gebruiken. Sla het resultaat op in de variabele `similarity`

In [ ]:
from cf1 import create_similarity_matrix_cosine

# TODO


Test je uitwerking:

In [ ]:
answers.test_14(similarity)

We hebben te weinig films in onze similarity matrix zitten om op een zinnige manier ratings te kunnen voorspellen en aanbevelingen te kunnen doen. Je zou in principe de utility en similarity matrix voor de hele data set kunnen bepalen en dan kan je voorspellingen doen zoals je dat ook in de vorige modules hebt gedaan. 

Je kan alleen de prestaties (mse, precision, recall, etc.) van het algorithme niet direct vergelijken met die uit de vorige opdrachten. De dataset bevat niet de ondertitels voor alle films uit `ratings.csv`.

### Vraag 9
\[2 pt.\]

a) Waarom kan je dan de prestaties niet direct vergelijken?

b) Wat zou je moeten doen om ervoor te zorgen dat je de prestaties van dit algoritme wel goed kan vergelijken met die van eerdere algoritmes (gegeven dat je niet _meer_ ondertitelingen kan vinden)?

YOUR ANSWER HERE

TF_IDF vectorisatie is slechts een manier om natuurlijke taal data te vectoriseren. SpaCy zelf heeft ook een ingebouwde aanpak hiervoor. Deze maakt gebruik van word2vec. Een vectorisatie die gebaseerd is op neurale netwerken. Hier ga je in de volgende module naar kijken.